In [2]:
from pandasql import sqldf
import pandas as pd
import json

In [3]:
orders = pd.read_csv('orders.csv')
sales = pd.read_csv('sales.csv')
user = pd.read_csv('user.csv')
store = pd.read_csv('store.csv')
product = pd.read_csv('product.csv')
orders = orders[:len(orders) // 100]

In [4]:
# В данных столбец orders['product_info'] представлен в некорректном формате. Формат JSON предполагает, что строка
# заключена в двойные кавычки, а не одинарные, а также, True и False должны быть написаны с маленькой буквы
# иначе мы просто не сможем корректно считать json-строку. Исправим это:
orders['product_info'] = orders['product_info'].replace('\'', '"', regex=True).replace('True', 'true', regex=True).replace('False', 'false', regex=True)

In [5]:
query = '''
/*
Считываем информацию из колонки product_info.
ВАЖНО: В колонке product_info лежат поломанные JSON'ы, не соответствующие правильному формату.
Формат JSON предполагает, что строка заключена в двойные кавычки, а не одинарные, а также, True и False 
должны быть написаны с маленькой буквы, иначе мы просто не сможем корректно считать json-строку.
Чтобы это починить после считывания данных в pandas добавляем строку:
orders['product_info'] = orders['product_info'].replace('\'', '"', regex=True).replace('True', 'true', regex=True).replace('False', 'false', regex=True)
*/
WITH product_info AS (
SELECT
    order_id,
    json_extract(value, '$.product_id') AS product_id,
    json_extract(value, '$.product_price') AS product_price,
    json_extract(value, '$.product_comission') AS product_comission,
    json_extract(value, '$.count') AS count
FROM
    orders,
    json_each(orders.product_info)
),
/*
Уберём лишние столбцы из таблицы sales, чтобы они не мешали JOIN'ам
*/
s AS (
SELECT
    order_id,
    product_id,
    is_canceled,
    is_accepted
FROM sales
)
    
SELECT
    o.user_id,
    o.order_id,
    o.order_date,
    SUM(p.product_price * p.count) AS order_sum,
    SUM(p.product_comission * p.count) AS order_revenue,
    SUM(p.count) AS order_quantity,
    COUNT(DISTINCT p.product_id) AS order_unique_products
FROM orders o
JOIN s ON o.order_id = s.order_id
JOIN product_info p ON o.order_id = p.order_id AND s.product_id = p.product_id
WHERE s.is_accepted = 1 AND s.is_canceled = 0
GROUP BY o.user_id, o.order_id, o.order_date
'''

result = sqldf(query)